In [1]:
import numpy as np
import matplotlib.pyplot as plt
np.seterr(divide='ignore', invalid='ignore')
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import Javascript, display


@interact(Hide_Code = True)
def code_toggle(Hide_Code):
    if Hide_Code:
        display(Javascript("Jupyter.notebook.execute_cells([3])"))
        display(Javascript("Jupyter.notebook.execute_cells([1])"))
    else:
        display(Javascript("Jupyter.notebook.execute_cells([3])"))
        display(Javascript("Jupyter.notebook.execute_cells([2])"))

interactive(children=(Checkbox(value=True, description='Hide_Code'), Output()), _dom_classes=('widget-interact…

In [10]:
%%html
<style>
div.input {
    display:none;
}
</style>

In [3]:
%%html
<style>
div.input {
    display:inline;
}
</style>

In [8]:
%%html
<style>
div.input {
}
</style>

# Image resolution and Field of View example

In [5]:
# load in data
imdata = np.genfromtxt('data/kspace_imdata.csv', delimiter=',')

# generate k-space
kdata = np.fft.fftshift(np.fft.fft2(imdata))

# Define base resolution and FOV
FOV = 200 # units are mm
Nx, Ny = imdata.shape
dx = FOV/Nx # units are mm
grid = np.meshgrid(np.arange(-(FOV/2-dx),(FOV/2)+dx,dx), np.arange(-(FOV/2-dx),(FOV/2)+dx,dx))

# Define a grid of k-space sample locations
FOVk = 2*np.pi/(dx)
dk = 2*np.pi/FOV
kgrid = np.meshgrid(np.arange(-(FOVk/2-dk),(FOVk/2)+dk,dk), np.arange(-(FOVk/2-dk),(FOVk/2)+dk,dk))

## Use the sliders to change the k-space sampling
Let's set FOV and resolution only in one direction
There is an option to apply to both directions

There are three interactive controls below
1. The first slider below can be used to reduce the image resolution (i.e. increase the voxel size)
2. The second slider below reduces the *encoded* FOV by the factor shown, only integers allowed
3. The sliders apply only to one dimension in k-space. Tick the checkbox below to apply to both

In [6]:
@interact(reduce_resolution = (0.1,1,0.1),
          reduce_FOV = (1,4,1),
          resample_both_dimensions = True)
def kspace_enc(reduce_resolution, reduce_FOV, resample_both_dimensions):
    # based on above make a filter for k-space data
    kfilter1 = np.zeros(kdata.shape)
    kfilter2 = np.zeros(kdata.shape)
    ksample = np.arange(-(FOVk/2-dk),(FOVk/2)+dk,dk)
    kfilter1[:,np.where(abs(ksample)<((FOVk/2)*reduce_resolution))]=1
    kfilter2[:,0:-1:reduce_FOV]=1
    kfilter = kfilter1 * kfilter2

    if resample_both_dimensions:
        # also apply in AP direction
        kfilter = kfilter * kfilter.conj().transpose()
    print("Total fraction of k-space sampled = {}".format(100*np.size(np.where(kfilter != 0))/(Nx*Ny)))
    
    kdata_sampled = kfilter * kdata
     # invert FT
    imencoded = np.fft.ifft2(np.fft.ifftshift(kdata_sampled))
    
    fig, axes = plt.subplots(1, 2, figsize=(8, 4))
    ax = axes.ravel()
    ax[0].imshow(abs(kdata_sampled), cmap='gray', extent=[-(FOVk/2-dk), (FOVk/2)+dk, -(FOVk/2-dk), (FOVk/2)+dk],vmin=0, vmax=10)
    ax[0].set_xlabel('k_x (rad/mm)')
    ax[0].set_ylabel('k_y (rad/mm)')
    ax[0].set_title('K-space Data')
    ax[1].imshow(abs(imencoded), cmap='gray', extent=[0, 1/reduce_FOV, 0, 1/reduce_FOV])
    ax[1].set_title('Reconstructed image')
    ax[1].axis('off')

    fig.tight_layout()
    plt.show()

interactive(children=(FloatSlider(value=0.5, description='reduce_resolution', max=1.0, min=0.1), IntSlider(val…